# Applied Project in Big Data on Industrial Dataset

## DATA COLLECTION TECHNIQUES
## Part I. Before models: data preprocessing

### 1. Libraries and Spark setup

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

We need to read many files at once, I would prefer Spark for it. We need `Spark environment` to run the code below.

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf, struct, count_distinct, from_unixtime, col

In [ ]:
conf = SparkConf()
conf.set('spark.master', 'local[*]')
conf.set('spark.driver.memory', '12G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

### 2. Data preprocessing: articles

In [ ]:
files_path = '../topic_2/articles_data'

In [ ]:
os.listdir(files_path)

In [ ]:
files_mask = f'{files_path}/*.json'
sdf = spark.read.json(files_mask)

In [ ]:
sdf.limit(2).toPandas()

In [ ]:
sdf = sdf.withColumn('file', F.input_file_name())
sdf.limit(2).toPandas()

In [ ]:
sdf = sdf.withColumn(
    'file', 
    F.regexp_replace(
        'file', 
        'file:///home/jovyan/apbdid_23/topic_2/articles_data/articles_lbl_', 
        '')
)
sdf = sdf.withColumn(
    'label', 
    F.regexp_replace(
        'file', 
        '.json', 
        '')
)
sdf.limit(2).toPandas()

In [ ]:
sdf = sdf.select(
    sdf.label,
    F.explode(sdf.articles)
)
sdf.limit(5).toPandas()

In [ ]:
sdf.printSchema()

In [ ]:
def flat_df(df, prefix=None):
    """
    Makes nested Spark dataframe flat.
    `prefix` is for naming columns
    that are uppacked.
    """
    flat_cols = [c[0] for c in df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in df.dtypes if c[1][:6] == 'struct']
    flat_df = df.select(
        flat_cols + 
        [F.col(ncol + '.' + col).alias(prefix + col if prefix else ncol + '_' + col ) 
         for ncol in nested_cols 
         for col in df.select(ncol + '.*').columns]
    )
    return flat_df

In [ ]:
print(flat_df.__doc__)

In [ ]:
sdf = flat_df(sdf, prefix='')
sdf.limit(5).toPandas()

In [ ]:
sdf_ds = sdf.select(
    sdf.label,
    sdf.col_name.alias('title'),
    sdf.col_annotation.alias('annotation')
)
sdf_ds.limit(5).toPandas()

In [ ]:
sdf_ds = sdf_ds.withColumn(
    'title', 
    F.regexp_replace(
        'title', 
        ';', 
        ',')
)
sdf_ds = sdf_ds.withColumn(
    'annotation', 
    F.regexp_replace(
        'annotation', 
        ';', 
        ',')
)

In [ ]:
sdf_ds.groupBy('label').count().show()

In [ ]:
sdf_ds.coalesce(1).write.csv('articles.csv', sep=';', header=True)

### 3. Data preprocessing: AI jobs

In [ ]:
files_path = '../topic_2/ai_jobs_data'

In [ ]:
len(os.listdir(files_path))

In [ ]:
os.listdir(files_path)[:3]

In [ ]:
files_mask = f'{files_path}/*.json'
sdf = spark.read.json(files_mask)

In [ ]:
sdf.printSchema()

In [ ]:
sdf.describe()

In [ ]:
sdf.limit(5).toPandas()

#### 3.1. Understading the target

In [ ]:
sdf = flat_df(sdf, prefix='')
sdf.limit(2).toPandas()

In [ ]:
# once againg to unpack all inner structures
sdf = flat_df(sdf, prefix='')
sdf.limit(2).toPandas()

In [ ]:
# step 1: split `salary range` column simply by space
split_col = F.split(sdf['salary_range'], ' ')

# step 2: extract values from string into columns
sdf = sdf.withColumn('currency', split_col.getItem(0))
sdf = sdf.withColumn('salary_value_min', split_col.getItem(1))
sdf = sdf.withColumn('salary_value_max', split_col.getItem(3))

# step 3: convert to numbers
sdf = sdf.withColumn('salary_value_min', 
                     F.regexp_replace('salary_value_min', 'K', ''))
sdf = sdf.withColumn('salary_value_min', col('salary_value_min') * 1000)
sdf = sdf.withColumn('salary_value_max', 
                     F.regexp_replace('salary_value_max', 'K', ''))
sdf = sdf.withColumn('salary_value_max', col('salary_value_max') * 1000)

In [ ]:
sdf.limit(2).toPandas()

In [ ]:
sdf = sdf.withColumn(
    'benefits_str',
    F.array_join(F.col('benefits'), ',')
)
sdf = sdf.withColumn(
    'skills_str',
    F.array_join(F.col('skills'), ',')
)

#### 6.2. Dataset for modelling

In [ ]:
sdf_ds = sdf.select(
    # these are target variables
    sdf.currency.alias('cur_1'),
    sdf.salary_value_min.alias('salary_min_1'),
    sdf.salary_value_max.alias('salary_max_1'),
    sdf.baseSalary_currency.alias('cur_2'),
    sdf.baseSalary_value_minValue.alias('salary_min_2'),
    sdf.baseSalary_value_maxValue.alias('salary_max_2'),
    sdf.baseSalary_value_unitText.alias('salary_period'),
    # that will be predictors
    sdf.company,
    sdf['hiringOrganization_@type'].alias('company_type'),
    sdf.jobLocation_address_addressCountry.alias('country'),
    sdf.location,
    sdf.position,
    sdf.level,
    sdf.employmentType.alias('type'),
    sdf.benefits_str.alias('benefits'),
    sdf.skills_str.alias('skills'),
    sdf.description
).dropDuplicates()
sdf_ds.limit(5).toPandas()

In [ ]:
sdf_ds.printSchema()

In [ ]:
sdf_ds.coalesce(1).write.csv('ai_jobs.csv', sep=';', header=True)